## Import modules

In [1]:
import pickle
import json

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorchtools
import glob as gl
import random
import os
import time
import itertools

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.datasets import make_regression
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import log_loss
from numpy import hstack
from numpy import vstack

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import warnings
warnings.filterwarnings(action='ignore')

## Data loading

In [2]:
def find_directory(foldername, filename = None, back_num = 0):
    cur = os.getcwd()
    for i in range(back_num):
        cur = os.path.abspath(os.path.join(cur, os.pardir))
    for folder in foldername:
        cur = os.path.join(cur, folder)
    if not os.path.exists(cur):
        os.makedirs(cur)
        print(f'{cur} created')
    if filename != None:
        cur = os.path.join(cur, filename)
    return cur

os.getcwd()
find_directory(back_num = 1, foldername = ['Dataset'], filename = 'bat_dict.pkl')

'C:\\Users\\User\\Desktop\\LIB\\CNN work\\Variable time length\\All dataset'

'C:\\Users\\User\\Desktop\\LIB\\CNN work\\Variable time length\\Dataset\\bat_dict.pkl'

In [3]:
bat_dict_add_1_2 = find_directory(back_num = 2, foldername = ['Dataset'], filename = 'bat_1_2_dict_VITQcQd.pkl')
with open(bat_dict_add_1_2, 'rb') as tf:
    bat_dict_1_2 = pickle.load(tf)

bat_sel_dict = bat_dict_1_2
max_time = 120

In [4]:
# Tensor dimension: (cell, cycle, time, variable)
num_cells = len(bat_sel_dict.keys())
regression_cycles = [45, 65, 85, 100]

# variable = [Qc, I, V, T, Qd]
num_variables = len(bat_sel_dict['b1c0']['1']['1.0'].keys())

for reg in regression_cycles:
    #'''RNN 가변길이 다룰수 있게?? 고민필요'''
    globals()[f'x_tensor_{reg}'] = torch.zeros(num_cells, reg, max_time, num_variables)
    cell_index = 1
    for cell in bat_sel_dict.keys():
        for cycle in range(1, reg+1):
            for time in bat_sel_dict[cell][str(cycle)].keys():
    #             print(bat_sel_dict[cell][str(cycle)][str(time)])
                value_list = [i for i in bat_sel_dict[cell][str(cycle)][str(time)].values()]
    #             print(value_list)
                globals()[f'x_tensor_{reg}'][cell_index-1, cycle-1, int(float(time)*max_time/60), :] = torch.FloatTensor(value_list)

        cell_index += 1

    globals()[f'x_tensor_{reg}'].size()

num_variables = x_tensor_100.shape[3]
time_length = x_tensor_100.shape[2]
# (cycle_index-1, cycle-1, time-1, variable(Qc, I, V, T, Qd))
# x_tensor[123, 99, 30, :]

torch.Size([124, 45, 120, 5])

torch.Size([124, 65, 120, 5])

torch.Size([124, 85, 120, 5])

torch.Size([124, 100, 120, 5])

In [5]:
y_df_add = find_directory(back_num = 2, foldername = ['Dataset', 'y_df.csv'])
y_df = pd.read_csv(y_df_add)

# 2. Regression problem
# KO, KP, CL
y_cl_numpy = y_df['Cycle life'].to_numpy()
y_kp_numpy = y_df['kneepoints'].to_numpy()
y_ko_numpy = y_df['kneeonsets'].to_numpy()
# y_ko_numpy.shape

y_regression_numpy = y_ko_numpy

y_ko_tensor = torch.FloatTensor(y_ko_numpy)
y_ko_tensor.size()

torch.Size([124])

In [6]:
def setRandomSeed(random_seed=0):
    os.environ['PYTHONHASHSEED'] = str(random_seed)
    torch.manual_seed(random_seed) # torch 
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
    torch.backends.cudnn.deterministic = True # cudnn
    torch.backends.cudnn.benchmark = False # cudnn
    np.random.seed(random_seed) # numpy
    random.seed(random_seed) # random

In [7]:
x_tensor_100.shape

torch.Size([124, 100, 120, 5])

### Dataloading as 2D CNN's method

In [8]:
# y: no scaling
class dataPrep_RNN_CNN(Dataset):  
    def __init__(self, x_tensor, y_tensor, batch_size, scaler, purpose : str, test_split: float):
        setRandomSeed(random_seed = 100)
        self.xdata = torch.permute(x_tensor, (0, 3, 2, 1))
        self.ydata = y_tensor
        
        self.batch_size = batch_size
                
        self.xscaler = scaler()
        self.purpose = purpose
        
        self.test_split = test_split
        
        self.train_size, self.val_size, self.test_size = self.get_splits()
        
        # Tensor dimension: (cell, variable, time, cycle)
        # If problem emerges, check this part first.
        for i in range(len(self.xdata)):
            for j in range(self.xdata.shape[1]):
                temp = np.expand_dims(self.xscaler.fit_transform(self.xdata[i,j,:,:]), axis=0)
                if j ==0:
                    temp2 = temp
                else:
                    temp2 = np.vstack((temp2,temp))
            temp2 = np.expand_dims(temp2, axis=0)
            if i==0:
                self.xdata_scaled = temp2
            else:
                self.xdata_scaled = np.vstack((self.xdata_scaled, temp2))

        self.ydata_scaled = self.ydata
        
        self.x_train_scaled, self.x_test_scaled, self.y_train_scaled, self.y_test_scaled = train_test_split(self.xdata_scaled, 
                                                                                                            self.ydata_scaled, 
                                                                                                            test_size = self.test_size, 
                                                                                                            shuffle = True, 
                                                                                                            random_state=100)
        
        self.x_train_scaled, self.x_val_scaled, self.y_train_scaled, self.y_val_scaled = train_test_split(self.x_train_scaled,
                                                                                                         self.y_train_scaled,
                                                                                                         test_size = self.val_size, 
                                                                                                         shuffle = True, 
                                                                                                         random_state = 100)
        
        print(self.x_train_scaled.shape, self.x_val_scaled.shape, self.x_test_scaled.shape,
              self.y_train_scaled.shape, self.y_val_scaled.shape, self.y_test_scaled.shape)
        
        self.train = list(zip(self.x_train_scaled, self.y_train_scaled))
        self.val = list(zip(self.x_val_scaled, self.y_val_scaled))
        self.test = list(zip(self.x_test_scaled, self.y_test_scaled))
                
        self.train_dataloader = DataLoader(self.train, batch_size = self.batch_size, shuffle=True)
        self.val_dataloader = DataLoader(self.val, batch_size = self.batch_size, shuffle=True)
        self.test_dataloader = DataLoader(self.test, batch_size = self.batch_size, shuffle=True)
            
    # split the datasets into training and testing
    def get_splits(self):
        # test size
        test_size = int(self.test_split*len(self.xdata))
        # val size
        val_size = int(0.2*(len(self.xdata) - test_size))
        # train_size
        train_size = len(self.xdata) - val_size - test_size
        return train_size, val_size, test_size
    
    # Returns dataloaders for training and validation datasets
    def train_data(self):
        return self.train_dataloader, self.val_dataloader
    
    # Returns dataloader for test datasets
    def test_data(self):
        return self.test_dataloader
    
    def scaler(self, purpose):
#         if purpose != 'classification':
#             return self.xscaler, self.yscaler
#         else:
        return self.xscaler
    
    # finding length of x 
    def __len__(self):
        return len(self.X)
    
    # indexing rows for calling
    def __getitem__(self, idx):
        return [self.X[idx], self.y[idx]]


In [9]:
# reg_dataset = dataPrep1D(x_tensor, y_ko_tensor, batch_size = 16, scaler = MinMaxScaler, purpose = 'regression', test_split = 0.2)
# reg_train_dataloader, reg_val_dataloader = reg_dataset.train_data()
# reg_test_dataloader = reg_dataset.test_data()
# reg_xscaler = reg_dataset.scaler('regression')

regression_cycles = [45, 65, 85, 100]
# regression_cycles = [100]


# x와 y가 1대1 대응이어야 dataloader를 만들 수 있어서 2D array 그대로 놓고 학습시 tensor transform 예정
for reg in regression_cycles:
    globals()[f'reg_dataset_{reg}'] = dataPrep_RNN_CNN(globals()[f'x_tensor_{reg}'], y_ko_tensor, batch_size = 4, scaler = MinMaxScaler, purpose =  'regression', test_split = 0.2)
    globals()[f'reg_train_dataloader_{reg}'], globals()[f'reg_val_dataloader_{reg}'] = globals()[f'reg_dataset_{reg}'].train_data()
    globals()[f'reg_test_dataloader_{reg}'] = globals()[f'reg_dataset_{reg}'].test_data()
    globals()[f'reg_xscaler_{reg}'] = globals()[f'reg_dataset_{reg}'].scaler('regression')

(80, 5, 120, 45) (20, 5, 120, 45) (24, 5, 120, 45) torch.Size([80]) torch.Size([20]) torch.Size([24])
(80, 5, 120, 65) (20, 5, 120, 65) (24, 5, 120, 65) torch.Size([80]) torch.Size([20]) torch.Size([24])
(80, 5, 120, 85) (20, 5, 120, 85) (24, 5, 120, 85) torch.Size([80]) torch.Size([20]) torch.Size([24])
(80, 5, 120, 100) (20, 5, 120, 100) (24, 5, 120, 100) torch.Size([80]) torch.Size([20]) torch.Size([24])


In [10]:
# train_features, train_labels = next(iter(cla_train_dataloader))
# print(f"Classification case_Inputs batch shape(Batch size, num_vars, timestep, cycles): {train_features.size()}")
# print(f"Classification case_Outputs batch shape(Batch size): {train_labels.size()}")

reg = 100
train_features, train_labels = next(iter(globals()[f'reg_train_dataloader_{reg}']))
print(f'reg = {reg}')
print(f"Regression case_Inputs batch shape(Batch size, num_vars, timestep, cycles): {train_features.size()}")
print(f"Regression case_Outputs batch shape(Batch size): {train_labels.size()}")

train_features.float().type()

reg = 100
Regression case_Inputs batch shape(Batch size, num_vars, timestep, cycles): torch.Size([4, 5, 120, 100])
Regression case_Outputs batch shape(Batch size): torch.Size([4])


'torch.FloatTensor'

## Define Model Class 

In [11]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [12]:
class RNNcell(nn.Module):
    def __init__(self, model, input_size, hidden_size, dropout = 0.1, bidirectional = False, num_layer=1):
        super(RNNcell, self).__init__()
        
        if torch.cuda.is_available():
            self.device = 'cuda'
        else:
            self.device = 'cpu'
            
        self.model = model
        self.hidden_size = hidden_size
        self.num_layer = num_layer
        self.bidirectional = bidirectional
        
        self.D = 1 + self.bidirectional
        self.batch_size = 4
        
        #Batch size * cycles, timestep, input_size
        if self.model.endswith('RNN'):
            self.rnn = nn.RNN(input_size, hidden_size, num_layer, batch_first = True, dropout = dropout, bidirectional = bidirectional).to(self.device)
        elif self.model.endswith('LSTM'):
            self.rnn = nn.LSTM(input_size, hidden_size, num_layer, batch_first = True, dropout = dropout, bidirectional = bidirectional).to(self.device)
        elif self.model.endswith('GRU'):
            self.rnn = nn.GRU(input_size, hidden_size, num_layer, batch_first = True, dropout = dropout, bidirectional = bidirectional).to(self.device)
            
        for name, param in self.rnn.named_parameters():
            if name.startswith('weight'):
                nn.init.xavier_uniform(param)
            else:
                nn.init.normal(param)
        
    def forward(self, x):
        # x made from (cell * n_cy, timestep(seq_len), num_vars)
        self.batch_size = x.size(0)
        
        D = 1 + self.bidirectional
        #out: Containing output features h_t from the last layer of LSTM for each t(if bidirectional ==True: concat of forward and backward hidden states)
        #h_n: Containing final hidden state in the sequence
        h0 = torch.zeros(D*self.num_layer, self.batch_size, self.hidden_size).to(self.device)
        if self.model.endswith('LSTM'):
            c0 = torch.zeros(D*self.num_layer, self.batch_size, self.hidden_size).to(self.device)
            # c_n: Containing final cell state in the sequence
            out, (hn, cn) = self.rnn(x, (h0, c0))
        else:
            out, hn = self.rnn(x, h0.detach())
        return out, hn

In [13]:
class SelfAttention(nn.Module):
    def __init__(self, hidden_size, num_heads):
        super(SelfAttention, self).__init__()
        self.hidden_size = hidden_size
        self.num_heads = num_heads
        self.head_size = hidden_size // num_heads
        
        self.query = nn.Linear(hidden_size, hidden_size)
        self.key = nn.Linear(hidden_size, hidden_size)
        self.value = nn.Linear(hidden_size, hidden_size)
        self.softmax = nn.Softmax(dim=-1)
        
    def forward(self, inputs):
        # inputs shape: [batch_size, seq_len, hidden_size]
        batch_size, seq_len, hidden_size = inputs.size()
        
        # Compute query, key, and value matrices
        query = self.query(inputs).view(batch_size, seq_len, self.num_heads, self.head_size).transpose(1, 2)
        key = self.key(inputs).view(batch_size, seq_len, self.num_heads, self.head_size).transpose(1, 2)
        value = self.value(inputs).view(batch_size, seq_len, self.num_heads, self.head_size).transpose(1, 2)
        
        # Compute scaled dot-product attention scores
        # attention_scores: [batch_size, num_heads, seq_len, seq_len]
        attention_scores = torch.matmul(query, key.transpose(-2, -1)) / torch.sqrt(torch.tensor(self.head_size, dtype=torch.float32))
        attention_probs = self.softmax(attention_scores)
        
        # Apply attention to the value matrix and concatenate heads
        # context: [batch_size, num_heads, seq_len, head_size]=>[batch_size, seq_len, num_heads, head_size] 
        # => [batch_size, seq_len, num_heads * head_size]
        context = torch.matmul(attention_probs, value).transpose(1, 2).contiguous().view(batch_size, seq_len, -1)
#         context = nn.Linear(hidden_size, 1)(context)
        return attention_probs, context

In [14]:
class RNN_TA_CA_1DCNN(nn.Module):
    def __init__(self, input_size, num_time, num_cycles, rnn1, bi1, hid1, nhead2, fil2, pool2, npool2, fsize2, psize2, mids, dr1 = 0.1, dr2 = 0.1, di2 = 1, st2 = 1, pad2 = 0):
        super(RNN_TA_CA_1DCNN, self).__init__()
        setRandomSeed()
        
        self.D1 = 1 + bi1
        
        self.rnn1 = RNNcell(rnn1, input_size, hid1, dr1, bi1)
        
        if torch.cuda.is_available():
            self.device = 'cuda'
        else:
            self.device = 'cpu'
        
        # Overall parameters
        self.num_time = num_time
        self.num_cycles = num_cycles
        self.mids = mids
        
        # Pooling/Nonpooling parameters
        # Pooling/Nonpooling layers
        self.pool2 = pool2
        self.npool2 = npool2
        # Pooling size(Pooling layer)
        self.psize2 = psize2
        
        # Params for self-attention(ca)
        self.nh2 = nhead2
        
        # CNN hyperparameters
        self.fil2 = fil2
        self.fsize2 = fsize2
        self.di2 = di2
        self.st2 = st2
        # Padding for CNN
        self.pad2 = pad2
                
        # Critical values for the 1st CNN layer
        # Batch for different cells
        
        # Conv1d takes input dimension: [Batch_size, c_in(no. of in_channels )= hidden_size, l_in = num_cycles]
        self.in2 = self.D1*hid1
        self.out2 = self.fil2
        self.Lout = self.num_cycles
        
        # Conv_block1~P
        # Pooling layers
        for i in range(1, self.pool2+1):
#             print(f"i: {i}, out_channels: {self.out_channels}")
            globals()[f'self.conv_block{i}'] = nn.Sequential(
            nn.Conv1d(self.in2, self.out2, self.fsize2, self.st2, self.pad2),
            nn.BatchNorm1d(self.out2),
            nn.ReLU(),
            nn.MaxPool1d(self.psize2)
            ).to(self.device)

            self.in2 = self.out2
            self.out2 = self.out2*2
            self.Lout = int(int((self.Lout+2*self.pad2-self.di2*(self.fsize2-1)-1)/self.st2+1)/self.psize2)

        for j in range(self.pool2+1, self.pool2+self.npool2+1):
            globals()[f'self.conv_block{j}'] = nn.Sequential(
            nn.Conv1d(self.in2, self.out2, self.fsize2, self.st2, self.pad2),
            nn.BatchNorm1d(self.out2),
            nn.ReLU(),
            ).to(self.device)
            
            self.in2 = self.out2
            self.out2 = self.out2*2
            self.Lout = int((self.Lout+2*self.pad2-self.di2*(self.fsize2-1)-1)/self.st2+1)
        
        self.lin1 = nn.Linear(self.D1*hid1, 1)
        self.sm1 = nn.Softmax(dim = 2)
        self.relu = nn.ReLU()
        
        self.ca = SelfAttention(self.D1*hid1, self.nh2)
        
        self.fc = nn.Sequential(nn.Linear(self.in2*self.Lout, self.mids[0], bias=True), nn.ReLU(),
                       nn.Linear(self.mids[0], self.mids[1], bias = True), nn.ReLU(),
                       nn.Linear(self.mids[1], self.mids[2], bias = True))
        
        self.fc.apply(self.init_weights)
        
    def init_weights(self, m):
        self.m = m
        if type(self.m) == nn.Linear:
            nn.init.xavier_uniform_(self.m.weight)
        
    def forward(self, x):
        # Original: Batch size, num_vars, timestep, cycles=>Batch size * cycles, timestep, input_size
        x = torch.reshape(x, (x.size(0)*x.size(3), x.size(2), x.size(1))).to(device)
#         print("Batch size * cycles, timestep, input_size: ", x.size())
        out1, hn1 = self.rnn1(x)
        
        # hn의 batch size가 n_cy*k로 설정
        cell_batch_size = int(hn1.size(1)/self.num_cycles)

        # Reshape for input
        # out1: (batch_size = num_cell*num_cycle, timestep, D1*hid1)
        # outs : (batch_size(num_cell), num_cycle, timestep, D1*hid1)
        outs1 = torch.reshape(out1, (cell_batch_size, -1, out1.size(1), out1.size(2)))
        # ta: (batch_size, num_cycle, timestep, 1(D1*hid1->1))
        ta = self.sm1(self.lin1(outs1))
        # ta_outs: (batch_size, num_cycle, timestep, D1*hid1)=>(batch_size, D1*hid1, num_cycle, timestep)
        ta_outs = torch.permute(ta*outs1, (0, 3, 1, 2))
        # ct_vec: (batch_size, D1*hid1, num_cycle)
        ct_vec = torch.sum(ta_outs, -1)
        # ct_vec shape: [batch_size, num_cycle, D1*hid1(hidden_size)]
        # hidden_size = num_heads * head_size
        # ct_vec transform back: [batch_size, num_heads, num_cycle, head_size]=>[batch_size, num_cycle, num_heads, head_size] 
        # => [batch_size, num_cycle, num_heads * head_size(hidden_size)]
        # ca: matmul(query[batch_size, num_heads, num_cycle, head_size], key^T[batch_size, num_heads, head_size, num_cycle])
        # ca shape: [batch_size, num_heads, num_cycle, num_cycle]
        ca, ct_vec = self.ca(ct_vec.transpose(2, 1))
#         print(f"After self attention: ca.size: {ca.size()}, ct_vec.size: {ct_vec.size()}")
        # ct_vec shape: [batch_size, D1*hid1, num_cycle]
        ct_vec = ct_vec.transpose(2, 1)
        
        
        # input for 1d cnn: (batch_size, D1*hid1, num_cycle)
        for i in range(1, self.pool2+self.npool2+1):
            ct_vec = globals()[f'self.conv_block{i}'](ct_vec)
        # ct_vec from final convolutional layer: (batch_size, self.in2*self.Lout)
        ct_vec = ct_vec.view(ct_vec.size(0), -1)
        # final_out: (batch_size, 1)
        final_out = self.fc(ct_vec)
        
        return final_out.squeeze(), ta.squeeze(), ca

In [15]:
# Function for saving and loading of training history
def save_data(D3_array, filename):
    with open(filename,"wb") as dat_:
        pickle.dump(D3_array,dat_)
        
def load_data(filename):
    with open(filename,"rb") as ld:
        x_temp = pickle.load(ld)
    return x_temp

# Early stopping class
class EarlyStopping:
    """주어진 patience 이후로 validation loss가 개선되지 않으면 학습을 조기 중지"""
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt'):
        """
        Args:
            patience (int): validation loss가 개선된 후 기다리는 기간
                            Default: 7
            verbose (bool): True일 경우 각 validation loss의 개선 사항 메세지 출력
                            Default: False
            delta (float): 개선되었다고 인정되는 monitered quantity의 최소 변화
                            Default: 0
            path (str): checkpoint저장 경로
                            Default: 'checkpoint.pt'
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.best_epoch = 1
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path

    def __call__(self, epoch, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.best_model = self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
#             print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.best_epoch = epoch
            self.best_model = self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''validation loss가 감소하면 모델을 저장한다.'''
        if self.verbose:
            print(f'At epoch {self.best_epoch}: Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss
        return model

In [16]:
class train:
    def __init__(self, model: nn.Module, train_dataloader, val_dataloader, test_dataloader, epoch: int, learning_rate=0.01, patience = 5, verbose = False):
        super().__init__()
        #   Reprocudtion
        setRandomSeed(100)
        random_seed = 100
        if torch.cuda.is_available():
            torch.cuda.manual_seed_all(random_seed)
            torch.backends.cudnn.deterministic = True
            torch.backends.cudnn.benchmark = False
        os.environ['PYTHONHASHSEED'] = str(random_seed)
        torch.manual_seed(random_seed)
        np.random.seed(random_seed)

        if torch.cuda.is_available():
            self.device = 'cuda'
        else:
            self.device = 'cpu'
        
        self.model = model.to(self.device)
        self.train_dataloader = train_dataloader
        self.val_dataloader = val_dataloader
        self.test_dataloader = test_dataloader
        self.epoch = epoch
        self.learning_rate = learning_rate
        
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr = self.learning_rate)
        self.early_stopping = EarlyStopping(patience = patience, verbose = verbose)

        self.criterion = nn.MSELoss()
        self.name = ['train_rmse', 'val_rmse', 'test_rmse']
        
    def EvalModel(self):
        EvalPredictions, EvalActuals = list(), list()
        TestPredictions, TestActuals = list(), list()
        self.model.eval()
        with torch.no_grad():
            for n, (inputs, targets) in enumerate(self.val_dataloader):
                inputs = inputs.float().to(self.device)
                targets = targets.float().to(self.device)

                yhat, ta, ca = self.model(inputs)

                if self.device !='cpu':
                    yhat = yhat.to('cpu')
                    targets = targets.to('cpu')    

                yhat = yhat.detach().numpy()
                actual = targets.numpy()
                EvalPredictions.append(yhat)
                EvalActuals.append(actual)

            EvalPredictions, EvalActuals = vstack(EvalPredictions), vstack(EvalActuals)
            val_rmse = mean_squared_error(EvalActuals, EvalPredictions, squared = False)
            
            for n, (inputs, targets) in enumerate(self.test_dataloader):
                inputs = inputs.float().to(self.device)
                targets = targets.float().to(self.device)

                yhat, ta, ca = self.model(inputs)

                if self.device !='cpu':
                    yhat = yhat.to('cpu')
                    targets = targets.to('cpu')    

                yhat = yhat.detach().numpy()
                actual = targets.numpy()
                TestPredictions.append(yhat)
                TestActuals.append(actual)

            TestPredictions, TestActuals = vstack(TestPredictions), vstack(TestActuals)
            test_rmse = mean_squared_error(TestActuals, TestPredictions, squared = False)
            
            return val_rmse, test_rmse

    def TrainModel(self):
        loss_history = []
        self.model.train()
        for i in range(self.epoch):
            predictions, actuals = list(), list()
            for n, (inputs, targets) in enumerate(self.train_dataloader):
                self.model.train()
                inputs = inputs.float().to(self.device)
                targets = targets.float().to(self.device)

                self.optimizer.zero_grad()

                yhat, ta, ca = self.model(inputs)
                    
                loss = self.criterion(yhat, targets)
                loss.backward()
                self.optimizer.step()
                
                if self.device !='cpu':
                    yhat = yhat.to('cpu')
                    targets = targets.to('cpu')
                    
                yhat = yhat.detach().numpy()
                predictions.append(yhat)
                
                targets = targets.numpy()
                actuals.append(targets)

            # evaluation of entire train data
            predictions, actuals = vstack(predictions), vstack(actuals)
            
            train_rmse = mean_squared_error(actuals, predictions, squared = False)

            # evaluation on validation data
            val_rmse, test_rmse = self.EvalModel()
            early_factor = val_rmse
            
            # training loss history
            loss_history.append([train_rmse, val_rmse, test_rmse])

            self.early_stopping(i+1, early_factor, self.model)
            
            if self.early_stopping.early_stop:
                print("Early stopping at best epoch: ", self.early_stopping.best_epoch)
                self.best_loss = loss_history[self.early_stopping.best_epoch-1]
                loss_history = loss_history[:self.early_stopping.best_epoch]
                
                return self.model, self.best_loss, pd.DataFrame(loss_history, columns = self.name), ta, ca
            
        print(f"Ends at final epoch {self.epoch}")
        print(f"Best epoch: {self.early_stopping.best_epoch}")
        self.best_loss = loss_history[self.early_stopping.best_epoch-1]
        loss_history = loss_history[:self.early_stopping.best_epoch]
        
        return self.model, self.best_loss, pd.DataFrame(loss_history, columns = self.name), ta, ca
        
    def predict(self, data: torch.Tensor):
        return self.model(data)
    
    def train_val_test_plot_rmse(self, num_cycles, modelname, n_ep, patience, rnn1, hid1, fil2, pool2, npool2, fsize2, psize2, lr):
        dataloaders = [self.train_dataloader, self.val_dataloader, self.test_dataloader]

        with torch.no_grad():
            model.eval()

            model_rmse = list()
            dset_index = 0
            for dataloader in dataloaders:
                if dset_index ==0:
                    dset = 'train'
                elif dset_index == 1:
                    dset = 'val'
                else:
                    dset = 'test'

                figadd = find_directory(back_num = 0, foldername = [f'{num_cycles} cycles', f'Depth Test_col_{n_ep}_{patience}', modelname, f'{rnn1}_1D CNN', 'train_history'], 
                                             filename = f'{rnn1}_hidden_{hid1}_n_fil_{fil2}_pool_{pool2}_npool_{npool2}_fsize_{fsize2}_psize_{psize2}_lr_1_{int(1/lr)}.jpg')

                predictions, actuals = list(), list()
                for i, (test_input, test_target) in enumerate(dataloader):
                    test_input = test_input.float().to(self.device)
                    test_target = test_target.float().to(self.device)
                    test_yhat, test_ta = model(test_input)

                    if self.device != 'cpu':
                        test_yhat = test_yhat.to('cpu')
                        test_target = test_target.to('cpu')

                    test_yhat = test_yhat.detach().tolist()
                    predictions += test_yhat
                    test_target = test_target.tolist()
                    actuals += test_target

                pred_acc, targ_acc = np.array(predictions), np.array(actuals)

                rmse = mean_squared_error(actuals, predictions, squared = False)
                model_rmse.append(rmse)

                plt.figure(figsize=[10,4])
                plt.rcParams["font.size"] = "12"
                plt.plot(pred_acc, label='Prediction')
                plt.plot(range(len(targ_acc)), targ_acc, label = 'Actual')
                plt.xticks(range(0, len(targ_acc) + 1, int(len(targ_acc)/5)))
                plt.title(f"{rnn1}_hid_{hid1}_nfil_{fil2}_pool_{pool2}_npool_{npool2}_fsize_{fsize2}_psize_{psize2}: RMSE={rmse}")
                plt.xlabel('cell')
                plt.ylabel('Lifespan(cycle)')
                plt.legend()
                plt.savefig(figadd)
                plt.close()

                dset_index += 1

        return model_rmse

## Implementation

In [17]:
def history_state_dict_add(num_cycles, modelname, n_ep, patience, rnn1, hid1, nh2, fil2, pool2, npool2, fsize2, psize2, lr):
    history_add = find_directory(back_num = 0, foldername = [f'{num_cycles} cycles', f'Depth Test_col_{n_ep}_{patience}', modelname, f'{rnn1}_1D CNN', 'train_history'], 
                                             filename = f'{rnn1}_hidden_{hid1}_nh_{nh2}_n_fil_{fil2}_pool_{pool2}_npool_{npool2}_fsize_{fsize2}_psize_{psize2}_lr_1_{int(1/lr)}.pkl')
    state_dict_add = find_directory(back_num = 0, foldername = [f'{num_cycles} cycles', f'Depth Test_col_{n_ep}_{patience}', modelname, f'{rnn1}_1D CNN', 'model'], 
                                             filename = f'{rnn1}_hidden_{hid1}_nh_{nh2}_n_fil_{fil2}_pool_{pool2}_npool_{npool2}_fsize_{fsize2}_psize_{psize2}_lr_1_{int(1/lr)}_state_dict.pth')
    ta_add = find_directory(back_num = 0, foldername = [f'{num_cycles} cycles', f'Depth Test_col_{n_ep}_{patience}', modelname, f'{rnn1}_1D CNN', 'ta'],
                                             filename = f'{rnn1}_hidden_{hid1}_nh_{nh2}_n_fil_{fil2}_pool_{pool2}_npool_{npool2}_fsize_{fsize2}_psize_{psize2}_lr_1_{int(1/lr)}_ta.wb')
    ca_add = find_directory(back_num = 0, foldername = [f'{num_cycles} cycles', f'Depth Test_col_{n_ep}_{patience}', modelname, f'{rnn1}_1D CNN', 'ca'],
                                             filename = f'{rnn1}_hidden_{hid1}_nh_{nh2}_n_fil_{fil2}_pool_{pool2}_npool_{npool2}_fsize_{fsize2}_psize_{psize2}_lr_1_{int(1/lr)}_ta.wb')
    return history_add, state_dict_add, ta_add, ca_add

In [18]:
# Hyperparameters
num_vars = train_features.size()[1]

modelname = 'RNN_TA_CA_1DCNN'
rnns = ['LSTM', 'GRU', 'RNN']
hids = [3, 5, 7]
num_time = max_time
num_cycles = 100
ep_pats = [[10000, 1000]]
lrs = [1e-4, 1e-3, 1e-2]

num_fils = [3, 5, 7]
pools = [1, 2]
npools = [1, 2]
fsize2 = 3
psize2 = 2
mids = [8, 4, 1]

nh2s = [5]

import time

for [n_ep, patience] in ep_pats:
    print(f"epoch = {n_ep}, patience = {patience}")
    trdl = globals()[f'reg_train_dataloader_{num_cycles}']
    vdl = globals()[f'reg_val_dataloader_{num_cycles}']
    tedl = globals()[f'reg_test_dataloader_{num_cycles}']
    for rnn1, hid1 in itertools.product(rnns, hids):
        bi1 = False
        
        if rnn1.startswith('Bi'):
            bi1 = True
        for pool2, npool2 in itertools.product(pools, npools):
            print(f'Pooling: {pool2} layers, Nonpooling: {npool2} layers, split by {60/max_time} min')
            for fil2, lr in itertools.product(num_fils, lrs):
                for nh2 in nh2s:
                    print(f"rnn = {rnn1}, bi = {bi1}, hid per head = {hid1}, num_heads = {nh2}, num_fil = {fil2}, lr = {lr}")
                    
                    all_hid1 = hid1 * nh2
                    
                    start = time.time()
                    
                    # Construct CNN
                    model = globals()[modelname](num_vars, num_time, num_cycles, rnn1, bi1, all_hid1, nh2, fil2, pool2, npool2, fsize2, psize2, mids).to(device)

                    # Train
                    model_train = train(model, trdl, vdl, tedl, n_ep, lr, patience, verbose = False)
                    best_model, best_loss, history, ta, ca = model_train.TrainModel()

                    print('best_loss = '+ str(best_loss))

                    history_add, state_dict_add, ta_add, ca_add = history_state_dict_add(num_cycles, modelname, n_ep, patience,
                                                                                rnn1, hid1, nh2, fil2, pool2, npool2, fsize2, psize2, lr)

                    # saving best_epoch, loss history, ta score
                    save_data(history, history_add)
                    save_data(best_model.state_dict(), state_dict_add)
                    save_data(ta, ta_add)
                    save_data(ca, ca_add)

                    print("time: ", time.time()-start)
                    print("\n\n")

epoch = 10000, patience = 1000
Pooling: 1 layers, Nonpooling: 1 layers, split by 0.5 min
rnn = LSTM, bi = False, hid per head = 3, num_heads = 5, num_fil = 3, lr = 0.0001
Early stopping at best epoch:  1495
best_loss = [131.11804, 77.67581, 80.68129]
C:\Users\User\Desktop\LIB\CNN work\Variable time length\All dataset\100 cycles\Depth Test_col_10000_1000\RNN_TA_CA_1DCNN\LSTM_1D CNN\train_history created
C:\Users\User\Desktop\LIB\CNN work\Variable time length\All dataset\100 cycles\Depth Test_col_10000_1000\RNN_TA_CA_1DCNN\LSTM_1D CNN\model created
C:\Users\User\Desktop\LIB\CNN work\Variable time length\All dataset\100 cycles\Depth Test_col_10000_1000\RNN_TA_CA_1DCNN\LSTM_1D CNN\ta created
C:\Users\User\Desktop\LIB\CNN work\Variable time length\All dataset\100 cycles\Depth Test_col_10000_1000\RNN_TA_CA_1DCNN\LSTM_1D CNN\ca created
time:  353.9454867839813



rnn = LSTM, bi = False, hid per head = 3, num_heads = 5, num_fil = 3, lr = 0.001
Early stopping at best epoch:  378
best_loss = [11

Early stopping at best epoch:  92
best_loss = [116.38674, 99.67677, 105.23097]
time:  157.02273750305176



rnn = LSTM, bi = False, hid per head = 5, num_heads = 5, num_fil = 5, lr = 0.0001
Early stopping at best epoch:  851
best_loss = [157.77975, 85.82746, 118.607056]
time:  267.0833921432495



rnn = LSTM, bi = False, hid per head = 5, num_heads = 5, num_fil = 5, lr = 0.001
Early stopping at best epoch:  316
best_loss = [132.58012, 89.67743, 73.296196]
time:  190.53269982337952



rnn = LSTM, bi = False, hid per head = 5, num_heads = 5, num_fil = 5, lr = 0.01
Early stopping at best epoch:  1877
best_loss = [33.179436, 88.15052, 92.35963]
time:  413.0859134197235



rnn = LSTM, bi = False, hid per head = 5, num_heads = 5, num_fil = 7, lr = 0.0001
Early stopping at best epoch:  2028
best_loss = [115.52002, 82.55663, 94.924644]
time:  434.21653723716736



rnn = LSTM, bi = False, hid per head = 5, num_heads = 5, num_fil = 7, lr = 0.001
Early stopping at best epoch:  290
best_loss = [10

Early stopping at best epoch:  283
best_loss = [63.073067, 105.64944, 95.93548]
time:  204.73391485214233



Pooling: 1 layers, Nonpooling: 2 layers, split by 0.5 min
rnn = LSTM, bi = False, hid per head = 7, num_heads = 5, num_fil = 3, lr = 0.0001
Early stopping at best epoch:  694
best_loss = [161.54953, 86.4855, 85.049385]
time:  278.62329173088074



rnn = LSTM, bi = False, hid per head = 7, num_heads = 5, num_fil = 3, lr = 0.001
Early stopping at best epoch:  307
best_loss = [137.08052, 94.87373, 68.23273]
time:  214.92401599884033



rnn = LSTM, bi = False, hid per head = 7, num_heads = 5, num_fil = 3, lr = 0.01
Early stopping at best epoch:  1611
best_loss = [22.376663, 79.47811, 102.45926]
time:  430.36412525177



rnn = LSTM, bi = False, hid per head = 7, num_heads = 5, num_fil = 5, lr = 0.0001
Early stopping at best epoch:  803
best_loss = [178.79842, 106.0783, 90.06414]
time:  296.8583881855011



rnn = LSTM, bi = False, hid per head = 7, num_heads = 5, num_fil = 5, lr = 0.0

Early stopping at best epoch:  4494
best_loss = [278.6232, 195.65594, 159.61874]
time:  838.1973707675934



rnn = GRU, bi = False, hid per head = 3, num_heads = 5, num_fil = 5, lr = 0.0001
Early stopping at best epoch:  1597
best_loss = [162.95972, 102.280106, 95.376724]
time:  395.4678223133087



rnn = GRU, bi = False, hid per head = 3, num_heads = 5, num_fil = 5, lr = 0.001
Early stopping at best epoch:  277
best_loss = [152.81659, 72.114044, 114.33615]
time:  195.19089722633362



rnn = GRU, bi = False, hid per head = 3, num_heads = 5, num_fil = 5, lr = 0.01
Early stopping at best epoch:  1694
best_loss = [41.30459, 87.283966, 92.82944]
time:  413.6827013492584



rnn = GRU, bi = False, hid per head = 3, num_heads = 5, num_fil = 7, lr = 0.0001
Early stopping at best epoch:  1209
best_loss = [181.6429, 90.81589, 99.956825]
time:  343.6932723522186



rnn = GRU, bi = False, hid per head = 3, num_heads = 5, num_fil = 7, lr = 0.001
Early stopping at best epoch:  318
best_loss = [114.9

Early stopping at best epoch:  1054
best_loss = [172.37753, 115.04589, 113.63646]
time:  297.93437147140503



rnn = GRU, bi = False, hid per head = 5, num_heads = 5, num_fil = 3, lr = 0.001
Early stopping at best epoch:  674
best_loss = [77.895996, 106.61926, 100.80817]
time:  242.27470874786377



rnn = GRU, bi = False, hid per head = 5, num_heads = 5, num_fil = 3, lr = 0.01
Early stopping at best epoch:  1620
best_loss = [21.736532, 95.97884, 83.04192]
time:  376.9141457080841



rnn = GRU, bi = False, hid per head = 5, num_heads = 5, num_fil = 5, lr = 0.0001
Early stopping at best epoch:  1543
best_loss = [149.92108, 114.875854, 169.67813]
time:  369.8801963329315



rnn = GRU, bi = False, hid per head = 5, num_heads = 5, num_fil = 5, lr = 0.001
Early stopping at best epoch:  154
best_loss = [151.28331, 112.57991, 112.9315]
time:  167.39821314811707



rnn = GRU, bi = False, hid per head = 5, num_heads = 5, num_fil = 5, lr = 0.01
Early stopping at best epoch:  1523
best_loss = [37.

Early stopping at best epoch:  199
best_loss = [119.13314, 104.109085, 91.14114]
time:  185.33025312423706



rnn = GRU, bi = False, hid per head = 7, num_heads = 5, num_fil = 7, lr = 0.01
Early stopping at best epoch:  1397
best_loss = [89.96714, 92.2106, 149.77025]
time:  368.1469645500183



Pooling: 2 layers, Nonpooling: 2 layers, split by 0.5 min
rnn = GRU, bi = False, hid per head = 7, num_heads = 5, num_fil = 3, lr = 0.0001
Early stopping at best epoch:  1384
best_loss = [135.66304, 110.28492, 98.99958]
time:  373.5676119327545



rnn = GRU, bi = False, hid per head = 7, num_heads = 5, num_fil = 3, lr = 0.001
Early stopping at best epoch:  986
best_loss = [50.92672, 112.50771, 71.64947]
time:  307.9570691585541



rnn = GRU, bi = False, hid per head = 7, num_heads = 5, num_fil = 3, lr = 0.01
Early stopping at best epoch:  398
best_loss = [140.33029, 93.101, 103.08037]
time:  216.9298176765442



rnn = GRU, bi = False, hid per head = 7, num_heads = 5, num_fil = 5, lr = 0.0001
Ear

Early stopping at best epoch:  825
best_loss = [54.910778, 66.58586, 85.77224]
time:  271.77041125297546



rnn = RNN, bi = False, hid per head = 3, num_heads = 5, num_fil = 3, lr = 0.01
Early stopping at best epoch:  12
best_loss = [187.45789, 117.43959, 101.85354]
time:  150.78263068199158



rnn = RNN, bi = False, hid per head = 3, num_heads = 5, num_fil = 5, lr = 0.0001
Early stopping at best epoch:  1555
best_loss = [132.73447, 100.97, 74.436485]
time:  389.9822025299072



rnn = RNN, bi = False, hid per head = 3, num_heads = 5, num_fil = 5, lr = 0.001
Early stopping at best epoch:  1021
best_loss = [36.258186, 82.4386, 90.50275]
time:  312.3946259021759



rnn = RNN, bi = False, hid per head = 3, num_heads = 5, num_fil = 5, lr = 0.01
Early stopping at best epoch:  1120
best_loss = [85.635284, 73.71905, 105.58982]
time:  325.517733335495



rnn = RNN, bi = False, hid per head = 3, num_heads = 5, num_fil = 7, lr = 0.0001
Early stopping at best epoch:  2045
best_loss = [104.10899, 7

Early stopping at best epoch:  921
best_loss = [36.42824, 97.829765, 86.96111]
time:  305.3850448131561



Pooling: 1 layers, Nonpooling: 1 layers, split by 0.5 min
rnn = RNN, bi = False, hid per head = 7, num_heads = 5, num_fil = 3, lr = 0.0001
Early stopping at best epoch:  2492
best_loss = [140.45978, 89.52276, 119.28946]
time:  524.0536088943481



rnn = RNN, bi = False, hid per head = 7, num_heads = 5, num_fil = 3, lr = 0.001
Early stopping at best epoch:  70
best_loss = [164.4894, 103.23398, 100.6086]
time:  161.14369773864746



rnn = RNN, bi = False, hid per head = 7, num_heads = 5, num_fil = 3, lr = 0.01
Early stopping at best epoch:  157
best_loss = [128.73993, 85.85779, 117.90958]
time:  174.98731780052185



rnn = RNN, bi = False, hid per head = 7, num_heads = 5, num_fil = 5, lr = 0.0001
Early stopping at best epoch:  2780
best_loss = [575.12604, 523.6655, 516.2741]
time:  574.7124485969543



rnn = RNN, bi = False, hid per head = 7, num_heads = 5, num_fil = 5, lr = 0.001
E

In [19]:
# Hyperparameters
num_vars = train_features.size()[1]

modelname = 'RNN_TA_CA_1DCNN'
rnns = ['LSTM', 'GRU', 'RNN']
hids = [3, 5, 7]
num_time = max_time
num_cycles = 100
ep_pats = [[10000, 1000]]
lrs = [1e-4, 1e-3, 1e-2]

num_fils = [3, 5, 7]
pools = [1, 2]
npools = [1, 2]
fsize2 = 3
psize2 = 2
mids = [8, 4, 1]

nh2s = [4]

import time

for [n_ep, patience] in ep_pats:
    print(f"epoch = {n_ep}, patience = {patience}")
    trdl = globals()[f'reg_train_dataloader_{num_cycles}']
    vdl = globals()[f'reg_val_dataloader_{num_cycles}']
    tedl = globals()[f'reg_test_dataloader_{num_cycles}']
    for rnn1, hid1 in itertools.product(rnns, hids):
        bi1 = False
        
        if rnn1.startswith('Bi'):
            bi1 = True
        for pool2, npool2 in itertools.product(pools, npools):
            print(f'Pooling: {pool2} layers, Nonpooling: {npool2} layers, split by {60/max_time} min')
            for fil2, lr in itertools.product(num_fils, lrs):
                for nh2 in nh2s:
                    print(f"rnn = {rnn1}, bi = {bi1}, hid per head = {hid1}, num_heads = {nh2}, num_fil = {fil2}, lr = {lr}")
                    
                    all_hid1 = hid1 * nh2
                    
                    start = time.time()
                    
                    # Construct CNN
                    model = globals()[modelname](num_vars, num_time, num_cycles, rnn1, bi1, all_hid1, nh2, fil2, pool2, npool2, fsize2, psize2, mids).to(device)

                    # Train
                    model_train = train(model, trdl, vdl, tedl, n_ep, lr, patience, verbose = False)
                    best_model, best_loss, history, ta, ca = model_train.TrainModel()

                    print('best_loss = '+ str(best_loss))

                    history_add, state_dict_add, ta_add, ca_add = history_state_dict_add(num_cycles, modelname, n_ep, patience,
                                                                                rnn1, hid1, nh2, fil2, pool2, npool2, fsize2, psize2, lr)

                    # saving best_epoch, loss history, ta score
                    save_data(history, history_add)
                    save_data(best_model.state_dict(), state_dict_add)
                    save_data(ta, ta_add)
                    save_data(ca, ca_add)

                    print("time: ", time.time()-start)
                    print("\n\n")

epoch = 10000, patience = 1000
Pooling: 1 layers, Nonpooling: 1 layers, split by 0.5 min
rnn = LSTM, bi = False, hid per head = 3, num_heads = 4, num_fil = 3, lr = 0.0001
Early stopping at best epoch:  1209
best_loss = [194.80772, 100.32121, 114.89493]
time:  330.1073338985443



rnn = LSTM, bi = False, hid per head = 3, num_heads = 4, num_fil = 3, lr = 0.001
Early stopping at best epoch:  232
best_loss = [147.88649, 96.30353, 74.89575]
time:  183.6900942325592



rnn = LSTM, bi = False, hid per head = 3, num_heads = 4, num_fil = 3, lr = 0.01
Early stopping at best epoch:  838
best_loss = [38.76131, 103.50911, 103.73951]
time:  274.6583478450775



rnn = LSTM, bi = False, hid per head = 3, num_heads = 4, num_fil = 5, lr = 0.0001
Early stopping at best epoch:  1543
best_loss = [158.20566, 79.919235, 127.98649]
time:  377.1539604663849



rnn = LSTM, bi = False, hid per head = 3, num_heads = 4, num_fil = 5, lr = 0.001
Early stopping at best epoch:  303
best_loss = [126.19822, 97.66292, 8

Early stopping at best epoch:  1994
best_loss = [144.54465, 107.41594, 112.112404]
time:  415.8024032115936



rnn = LSTM, bi = False, hid per head = 5, num_heads = 4, num_fil = 7, lr = 0.001
Early stopping at best epoch:  318
best_loss = [112.987785, 93.22747, 87.44879]
time:  183.76713681221008



rnn = LSTM, bi = False, hid per head = 5, num_heads = 4, num_fil = 7, lr = 0.01
Early stopping at best epoch:  481
best_loss = [25.064365, 90.65716, 74.96019]
time:  206.83001899719238



Pooling: 1 layers, Nonpooling: 2 layers, split by 0.5 min
rnn = LSTM, bi = False, hid per head = 5, num_heads = 4, num_fil = 3, lr = 0.0001
Early stopping at best epoch:  561
best_loss = [193.57257, 108.17996, 94.114815]
time:  223.79949378967285



rnn = LSTM, bi = False, hid per head = 5, num_heads = 4, num_fil = 3, lr = 0.001
Early stopping at best epoch:  1315
best_loss = [47.84436, 82.39692, 80.346565]
time:  331.1834170818329



rnn = LSTM, bi = False, hid per head = 5, num_heads = 4, num_fil = 3, lr

Early stopping at best epoch:  1193
best_loss = [133.32034, 94.49551, 81.238625]
time:  329.03340196609497



rnn = LSTM, bi = False, hid per head = 7, num_heads = 4, num_fil = 5, lr = 0.001
Early stopping at best epoch:  1453
best_loss = [24.8796, 68.40223, 110.16095]
time:  368.42410492897034



rnn = LSTM, bi = False, hid per head = 7, num_heads = 4, num_fil = 5, lr = 0.01
Early stopping at best epoch:  51
best_loss = [121.921646, 113.8813, 90.11458]
time:  158.27080702781677



rnn = LSTM, bi = False, hid per head = 7, num_heads = 4, num_fil = 7, lr = 0.0001
Early stopping at best epoch:  1212
best_loss = [217.97115, 104.75488, 98.245415]
time:  331.6248128414154



rnn = LSTM, bi = False, hid per head = 7, num_heads = 4, num_fil = 7, lr = 0.001
Early stopping at best epoch:  316
best_loss = [111.36147, 84.58123, 96.27556]
time:  200.62271094322205



rnn = LSTM, bi = False, hid per head = 7, num_heads = 4, num_fil = 7, lr = 0.01
Early stopping at best epoch:  92
best_loss = [107.2

Early stopping at best epoch:  1758
best_loss = [191.706, 112.34306, 97.54668]
time:  400.9935350418091



rnn = GRU, bi = False, hid per head = 3, num_heads = 4, num_fil = 3, lr = 0.001
Early stopping at best epoch:  2210
best_loss = [22.038881, 88.916435, 87.24102]
time:  469.8895938396454



rnn = GRU, bi = False, hid per head = 3, num_heads = 4, num_fil = 3, lr = 0.01
Early stopping at best epoch:  378
best_loss = [44.79613, 98.873375, 81.3955]
time:  203.4856915473938



rnn = GRU, bi = False, hid per head = 3, num_heads = 4, num_fil = 5, lr = 0.0001
Early stopping at best epoch:  890
best_loss = [190.6918, 119.87118, 138.66798]
time:  277.0219314098358



rnn = GRU, bi = False, hid per head = 3, num_heads = 4, num_fil = 5, lr = 0.001
Early stopping at best epoch:  1382
best_loss = [67.55705, 100.12995, 95.98422]
time:  345.83251881599426



rnn = GRU, bi = False, hid per head = 3, num_heads = 4, num_fil = 5, lr = 0.01
Early stopping at best epoch:  4494
best_loss = [278.6247, 195

Early stopping at best epoch:  354
best_loss = [128.96622, 108.306076, 85.87915]
time:  205.86357164382935



rnn = GRU, bi = False, hid per head = 5, num_heads = 4, num_fil = 7, lr = 0.01
Early stopping at best epoch:  675
best_loss = [23.119019, 110.13612, 87.48241]
time:  255.66827940940857



Pooling: 2 layers, Nonpooling: 2 layers, split by 0.5 min
rnn = GRU, bi = False, hid per head = 5, num_heads = 4, num_fil = 3, lr = 0.0001
Early stopping at best epoch:  659
best_loss = [206.2233, 123.9106, 184.80171]
time:  254.7146339416504



rnn = GRU, bi = False, hid per head = 5, num_heads = 4, num_fil = 3, lr = 0.001
Early stopping at best epoch:  1442
best_loss = [53.246857, 87.13953, 80.380745]
time:  370.7455904483795



rnn = GRU, bi = False, hid per head = 5, num_heads = 4, num_fil = 3, lr = 0.01
Early stopping at best epoch:  52
best_loss = [148.81873, 122.11287, 92.087814]
time:  160.50941920280457



rnn = GRU, bi = False, hid per head = 5, num_heads = 4, num_fil = 5, lr = 0.000

Early stopping at best epoch:  143
best_loss = [122.23245, 128.88037, 89.02721]
time:  142.79995274543762



rnn = GRU, bi = False, hid per head = 7, num_heads = 4, num_fil = 5, lr = 0.01
Early stopping at best epoch:  299
best_loss = [46.701813, 125.81972, 76.19807]
time:  162.1190311908722



rnn = GRU, bi = False, hid per head = 7, num_heads = 4, num_fil = 7, lr = 0.0001
Early stopping at best epoch:  1212
best_loss = [165.52559, 122.73776, 123.147415]
time:  278.9487473964691



rnn = GRU, bi = False, hid per head = 7, num_heads = 4, num_fil = 7, lr = 0.001
Early stopping at best epoch:  224
best_loss = [131.72488, 120.10426, 82.412224]
time:  154.07392144203186



rnn = GRU, bi = False, hid per head = 7, num_heads = 4, num_fil = 7, lr = 0.01
Early stopping at best epoch:  956
best_loss = [27.120537, 94.15039, 105.87486]
time:  245.83203053474426



Pooling: 1 layers, Nonpooling: 1 layers, split by 0.5 min
rnn = RNN, bi = False, hid per head = 3, num_heads = 4, num_fil = 3, lr = 0.

Early stopping at best epoch:  290
best_loss = [117.23325, 86.85068, 75.679924]
time:  177.90567564964294



rnn = RNN, bi = False, hid per head = 5, num_heads = 4, num_fil = 3, lr = 0.01
Early stopping at best epoch:  61
best_loss = [150.13548, 108.49414, 90.56853]
time:  146.1612093448639



rnn = RNN, bi = False, hid per head = 5, num_heads = 4, num_fil = 5, lr = 0.0001
Early stopping at best epoch:  894
best_loss = [132.98662, 84.63466, 98.95497]
time:  259.09948468208313



rnn = RNN, bi = False, hid per head = 5, num_heads = 4, num_fil = 5, lr = 0.001
Early stopping at best epoch:  993
best_loss = [54.23319, 102.02563, 105.26744]
time:  269.5574221611023



rnn = RNN, bi = False, hid per head = 5, num_heads = 4, num_fil = 5, lr = 0.01
Early stopping at best epoch:  1042
best_loss = [125.319595, 89.500656, 80.33899]
time:  275.7565755844116



rnn = RNN, bi = False, hid per head = 5, num_heads = 4, num_fil = 7, lr = 0.0001
Early stopping at best epoch:  1474
best_loss = [102.51056

Early stopping at best epoch:  459
best_loss = [81.80091, 89.84677, 108.45355]
time:  205.93318033218384



Pooling: 1 layers, Nonpooling: 2 layers, split by 0.5 min
rnn = RNN, bi = False, hid per head = 7, num_heads = 4, num_fil = 3, lr = 0.0001
Early stopping at best epoch:  1308
best_loss = [147.54933, 94.24675, 73.48016]
time:  335.21307277679443



rnn = RNN, bi = False, hid per head = 7, num_heads = 4, num_fil = 3, lr = 0.001
Early stopping at best epoch:  1415
best_loss = [40.101692, 73.28473, 90.27651]
time:  350.54807472229004



rnn = RNN, bi = False, hid per head = 7, num_heads = 4, num_fil = 3, lr = 0.01
Early stopping at best epoch:  162
best_loss = [125.45589, 98.292946, 87.477554]
time:  168.29961681365967



rnn = RNN, bi = False, hid per head = 7, num_heads = 4, num_fil = 5, lr = 0.0001
Early stopping at best epoch:  1212
best_loss = [195.29384, 102.53147, 105.119156]
time:  322.6512236595154



rnn = RNN, bi = False, hid per head = 7, num_heads = 4, num_fil = 5, lr = 

In [20]:
# Hyperparameters
num_vars = train_features.size()[1]

modelname = 'RNN_TA_CA_1DCNN'
rnns = ['LSTM', 'GRU', 'RNN']
hids = [3, 5, 7]
num_time = max_time
num_cycles = 100
ep_pats = [[10000, 1000]]
lrs = [1e-4, 1e-3, 1e-2]

num_fils = [3, 5, 7]
pools = [1, 2]
npools = [1, 2]
fsize2 = 3
psize2 = 2
mids = [8, 4, 1]

nh2s = [3]

import time

for [n_ep, patience] in ep_pats:
    print(f"epoch = {n_ep}, patience = {patience}")
    trdl = globals()[f'reg_train_dataloader_{num_cycles}']
    vdl = globals()[f'reg_val_dataloader_{num_cycles}']
    tedl = globals()[f'reg_test_dataloader_{num_cycles}']
    for rnn1, hid1 in itertools.product(rnns, hids):
        bi1 = False
        
        if rnn1.startswith('Bi'):
            bi1 = True
        for pool2, npool2 in itertools.product(pools, npools):
            print(f'Pooling: {pool2} layers, Nonpooling: {npool2} layers, split by {60/max_time} min')
            for fil2, lr in itertools.product(num_fils, lrs):
                for nh2 in nh2s:
                    print(f"rnn = {rnn1}, bi = {bi1}, hid per head = {hid1}, num_heads = {nh2}, num_fil = {fil2}, lr = {lr}")
                    
                    all_hid1 = hid1 * nh2
                    
                    start = time.time()
                    
                    # Construct CNN
                    model = globals()[modelname](num_vars, num_time, num_cycles, rnn1, bi1, all_hid1, nh2, fil2, pool2, npool2, fsize2, psize2, mids).to(device)

                    # Train
                    model_train = train(model, trdl, vdl, tedl, n_ep, lr, patience, verbose = False)
                    best_model, best_loss, history, ta, ca = model_train.TrainModel()

                    print('best_loss = '+ str(best_loss))

                    history_add, state_dict_add, ta_add, ca_add = history_state_dict_add(num_cycles, modelname, n_ep, patience,
                                                                                rnn1, hid1, nh2, fil2, pool2, npool2, fsize2, psize2, lr)

                    # saving best_epoch, loss history, ta score
                    save_data(history, history_add)
                    save_data(best_model.state_dict(), state_dict_add)
                    save_data(ta, ta_add)
                    save_data(ca, ca_add)

                    print("time: ", time.time()-start)
                    print("\n\n")

epoch = 10000, patience = 1000
Pooling: 1 layers, Nonpooling: 1 layers, split by 0.5 min
rnn = LSTM, bi = False, hid per head = 3, num_heads = 3, num_fil = 3, lr = 0.0001
Early stopping at best epoch:  2882
best_loss = [123.62572, 86.52274, 54.981018]
time:  552.1598761081696



rnn = LSTM, bi = False, hid per head = 3, num_heads = 3, num_fil = 3, lr = 0.001
Early stopping at best epoch:  290
best_loss = [130.668, 92.52564, 68.34883]
time:  184.00454807281494



rnn = LSTM, bi = False, hid per head = 3, num_heads = 3, num_fil = 3, lr = 0.01
Early stopping at best epoch:  52
best_loss = [151.67126, 121.18595, 78.34923]
time:  149.76149463653564



rnn = LSTM, bi = False, hid per head = 3, num_heads = 3, num_fil = 5, lr = 0.0001
Early stopping at best epoch:  1655
best_loss = [144.47903, 81.0087, 108.778244]
time:  378.0333733558655



rnn = LSTM, bi = False, hid per head = 3, num_heads = 3, num_fil = 5, lr = 0.001
Early stopping at best epoch:  382
best_loss = [115.87669, 95.891624, 85.

Early stopping at best epoch:  529
best_loss = [207.2727, 96.89261, 102.57597]
time:  222.36716389656067



rnn = LSTM, bi = False, hid per head = 5, num_heads = 3, num_fil = 7, lr = 0.001
Early stopping at best epoch:  173
best_loss = [160.88658, 85.37332, 100.52628]
time:  170.0383677482605



rnn = LSTM, bi = False, hid per head = 5, num_heads = 3, num_fil = 7, lr = 0.01
Early stopping at best epoch:  51
best_loss = [141.72748, 97.43366, 76.58185]
time:  152.6054391860962



Pooling: 1 layers, Nonpooling: 2 layers, split by 0.5 min
rnn = LSTM, bi = False, hid per head = 5, num_heads = 3, num_fil = 3, lr = 0.0001
Early stopping at best epoch:  1543
best_loss = [125.008865, 79.521835, 69.120544]
time:  379.10599756240845



rnn = LSTM, bi = False, hid per head = 5, num_heads = 3, num_fil = 3, lr = 0.001
Early stopping at best epoch:  211
best_loss = [135.75676, 78.73766, 91.55591]
time:  180.16914653778076



rnn = LSTM, bi = False, hid per head = 5, num_heads = 3, num_fil = 3, lr = 0

Early stopping at best epoch:  2511
best_loss = [144.115, 90.80742, 101.26675]
time:  522.2758829593658



rnn = LSTM, bi = False, hid per head = 7, num_heads = 3, num_fil = 5, lr = 0.001
Early stopping at best epoch:  1021
best_loss = [132.04904, 104.00223, 109.60297]
time:  300.1035406589508



rnn = LSTM, bi = False, hid per head = 7, num_heads = 3, num_fil = 5, lr = 0.01
Early stopping at best epoch:  195
best_loss = [58.720192, 110.78615, 68.29441]
time:  177.5954179763794



rnn = LSTM, bi = False, hid per head = 7, num_heads = 3, num_fil = 7, lr = 0.0001
Early stopping at best epoch:  300
best_loss = [221.41898, 110.491745, 92.56194]
time:  194.11810970306396



rnn = LSTM, bi = False, hid per head = 7, num_heads = 3, num_fil = 7, lr = 0.001
Early stopping at best epoch:  66
best_loss = [180.00188, 123.05592, 85.07459]
time:  159.33991384506226



rnn = LSTM, bi = False, hid per head = 7, num_heads = 3, num_fil = 7, lr = 0.01
Early stopping at best epoch:  37
best_loss = [121.15

Early stopping at best epoch:  851
best_loss = [183.33435, 109.37085, 147.23586]
time:  267.2681174278259



rnn = GRU, bi = False, hid per head = 3, num_heads = 3, num_fil = 3, lr = 0.001
Early stopping at best epoch:  143
best_loss = [159.10065, 108.44508, 97.81259]
time:  164.34245347976685



rnn = GRU, bi = False, hid per head = 3, num_heads = 3, num_fil = 3, lr = 0.01
Early stopping at best epoch:  92
best_loss = [107.91554, 124.447876, 88.24312]
time:  157.2395100593567



rnn = GRU, bi = False, hid per head = 3, num_heads = 3, num_fil = 5, lr = 0.0001
Early stopping at best epoch:  2263
best_loss = [118.155106, 95.53058, 86.98857]
time:  470.80107378959656



rnn = GRU, bi = False, hid per head = 3, num_heads = 3, num_fil = 5, lr = 0.001
Early stopping at best epoch:  156
best_loss = [183.62006, 107.852516, 87.790634]
time:  166.75248003005981



rnn = GRU, bi = False, hid per head = 3, num_heads = 3, num_fil = 5, lr = 0.01
Early stopping at best epoch:  675
best_loss = [33.084

Early stopping at best epoch:  70
best_loss = [179.20602, 103.147064, 105.64485]
time:  162.51475071907043



rnn = GRU, bi = False, hid per head = 5, num_heads = 3, num_fil = 7, lr = 0.01
Early stopping at best epoch:  1611
best_loss = [41.19314, 98.846275, 87.40841]
time:  398.3860535621643



Pooling: 2 layers, Nonpooling: 2 layers, split by 0.5 min
rnn = GRU, bi = False, hid per head = 5, num_heads = 3, num_fil = 3, lr = 0.0001
Early stopping at best epoch:  806
best_loss = [199.95041, 112.9432, 123.14451]
time:  282.390572309494



rnn = GRU, bi = False, hid per head = 5, num_heads = 3, num_fil = 3, lr = 0.001
Early stopping at best epoch:  343
best_loss = [96.05767, 104.23941, 87.86396]
time:  210.22054314613342



rnn = GRU, bi = False, hid per head = 5, num_heads = 3, num_fil = 3, lr = 0.01
Early stopping at best epoch:  1042
best_loss = [41.138496, 106.69852, 79.88068]
time:  321.31291127204895



rnn = GRU, bi = False, hid per head = 5, num_heads = 3, num_fil = 5, lr = 0.0001

Early stopping at best epoch:  2085
best_loss = [37.65994, 94.641945, 71.22202]
time:  489.1506712436676



rnn = GRU, bi = False, hid per head = 7, num_heads = 3, num_fil = 5, lr = 0.01
Early stopping at best epoch:  931
best_loss = [26.502447, 102.4642, 77.008545]
time:  310.32713890075684



rnn = GRU, bi = False, hid per head = 7, num_heads = 3, num_fil = 7, lr = 0.0001
Early stopping at best epoch:  806
best_loss = [187.78233, 119.00208, 126.3658]
time:  290.3556082248688



rnn = GRU, bi = False, hid per head = 7, num_heads = 3, num_fil = 7, lr = 0.001
Early stopping at best epoch:  766
best_loss = [78.36461, 94.12378, 98.22796]
time:  283.938884973526



rnn = GRU, bi = False, hid per head = 7, num_heads = 3, num_fil = 7, lr = 0.01
Early stopping at best epoch:  4494
best_loss = [278.62384, 195.65521, 159.61844]
time:  884.6095507144928



Pooling: 1 layers, Nonpooling: 1 layers, split by 0.5 min
rnn = RNN, bi = False, hid per head = 3, num_heads = 3, num_fil = 3, lr = 0.0001
Ea

Early stopping at best epoch:  191
best_loss = [124.95427, 94.38584, 58.735443]
time:  171.99147486686707



rnn = RNN, bi = False, hid per head = 5, num_heads = 3, num_fil = 3, lr = 0.01
Early stopping at best epoch:  798
best_loss = [90.07028, 76.95389, 65.323746]
time:  256.74501299858093



rnn = RNN, bi = False, hid per head = 5, num_heads = 3, num_fil = 5, lr = 0.0001
Early stopping at best epoch:  751
best_loss = [175.88287, 86.08909, 74.51607]
time:  251.99415516853333



rnn = RNN, bi = False, hid per head = 5, num_heads = 3, num_fil = 5, lr = 0.001
Early stopping at best epoch:  290
best_loss = [98.263916, 100.965836, 87.569954]
time:  186.64525198936462



rnn = RNN, bi = False, hid per head = 5, num_heads = 3, num_fil = 5, lr = 0.01
Early stopping at best epoch:  58
best_loss = [129.67445, 88.44306, 81.26338]
time:  153.34134459495544



rnn = RNN, bi = False, hid per head = 5, num_heads = 3, num_fil = 7, lr = 0.0001
Early stopping at best epoch:  561
best_loss = [169.0553,

Early stopping at best epoch:  403
best_loss = [43.20642, 113.619156, 84.05429]
time:  196.92510724067688



Pooling: 1 layers, Nonpooling: 2 layers, split by 0.5 min
rnn = RNN, bi = False, hid per head = 7, num_heads = 3, num_fil = 3, lr = 0.0001
Early stopping at best epoch:  751
best_loss = [197.50381, 96.9426, 122.48207]
time:  254.05341792106628



rnn = RNN, bi = False, hid per head = 7, num_heads = 3, num_fil = 3, lr = 0.001
Early stopping at best epoch:  194
best_loss = [126.246544, 99.867744, 78.66194]
time:  173.30326128005981



rnn = RNN, bi = False, hid per head = 7, num_heads = 3, num_fil = 3, lr = 0.01
Early stopping at best epoch:  700
best_loss = [97.50334, 108.35339, 89.17526]
time:  247.30070114135742



rnn = RNN, bi = False, hid per head = 7, num_heads = 3, num_fil = 5, lr = 0.0001
Early stopping at best epoch:  561
best_loss = [199.38295, 84.21983, 110.9299]
time:  228.22351360321045



rnn = RNN, bi = False, hid per head = 7, num_heads = 3, num_fil = 5, lr = 0.00

In [21]:
# Hyperparameters
num_vars = train_features.size()[1]

modelname = 'RNN_TA_CA_1DCNN'
rnns = ['LSTM', 'GRU', 'RNN']
hids = [3, 5, 7]
num_time = max_time
num_cycles = 100
ep_pats = [[10000, 1000]]
lrs = [1e-4, 1e-3, 1e-2]

num_fils = [3, 5, 7]
pools = [1, 2]
npools = [1, 2]
fsize2 = 3
psize2 = 2
mids = [8, 4, 1]

nh2s = [2]

import time

for [n_ep, patience] in ep_pats:
    print(f"epoch = {n_ep}, patience = {patience}")
    trdl = globals()[f'reg_train_dataloader_{num_cycles}']
    vdl = globals()[f'reg_val_dataloader_{num_cycles}']
    tedl = globals()[f'reg_test_dataloader_{num_cycles}']
    for rnn1, hid1 in itertools.product(rnns, hids):
        bi1 = False
        
        if rnn1.startswith('Bi'):
            bi1 = True
        for pool2, npool2 in itertools.product(pools, npools):
            print(f'Pooling: {pool2} layers, Nonpooling: {npool2} layers, split by {60/max_time} min')
            for fil2, lr in itertools.product(num_fils, lrs):
                for nh2 in nh2s:
                    print(f"rnn = {rnn1}, bi = {bi1}, hid per head = {hid1}, num_heads = {nh2}, num_fil = {fil2}, lr = {lr}")
                    
                    all_hid1 = hid1 * nh2
                    
                    start = time.time()
                    
                    # Construct CNN
                    model = globals()[modelname](num_vars, num_time, num_cycles, rnn1, bi1, all_hid1, nh2, fil2, pool2, npool2, fsize2, psize2, mids).to(device)

                    # Train
                    model_train = train(model, trdl, vdl, tedl, n_ep, lr, patience, verbose = False)
                    best_model, best_loss, history, ta, ca = model_train.TrainModel()

                    print('best_loss = '+ str(best_loss))

                    history_add, state_dict_add, ta_add, ca_add = history_state_dict_add(num_cycles, modelname, n_ep, patience,
                                                                                rnn1, hid1, nh2, fil2, pool2, npool2, fsize2, psize2, lr)

                    # saving best_epoch, loss history, ta score
                    save_data(history, history_add)
                    save_data(best_model.state_dict(), state_dict_add)
                    save_data(ta, ta_add)
                    save_data(ca, ca_add)

                    print("time: ", time.time()-start)
                    print("\n\n")

epoch = 10000, patience = 1000
Pooling: 1 layers, Nonpooling: 1 layers, split by 0.5 min
rnn = LSTM, bi = False, hid per head = 3, num_heads = 2, num_fil = 3, lr = 0.0001
Early stopping at best epoch:  1144
best_loss = [232.03482, 117.08364, 116.63687]
time:  312.21049904823303



rnn = LSTM, bi = False, hid per head = 3, num_heads = 2, num_fil = 3, lr = 0.001
Early stopping at best epoch:  2610
best_loss = [30.987965, 76.48958, 80.56516]
time:  527.9775850772858



rnn = LSTM, bi = False, hid per head = 3, num_heads = 2, num_fil = 3, lr = 0.01
Early stopping at best epoch:  667
best_loss = [33.44741, 107.102554, 90.996704]
time:  243.81490659713745



rnn = LSTM, bi = False, hid per head = 3, num_heads = 2, num_fil = 5, lr = 0.0001
Early stopping at best epoch:  2200
best_loss = [204.98248, 91.59233, 120.40692]
time:  468.45752215385437



rnn = LSTM, bi = False, hid per head = 3, num_heads = 2, num_fil = 5, lr = 0.001
Early stopping at best epoch:  401
best_loss = [122.27541, 98.1276

Early stopping at best epoch:  1769
best_loss = [139.15207, 86.18458, 71.28985]
time:  410.30000472068787



rnn = LSTM, bi = False, hid per head = 5, num_heads = 2, num_fil = 7, lr = 0.001
Early stopping at best epoch:  318
best_loss = [109.32707, 87.277466, 93.757256]
time:  194.13074231147766



rnn = LSTM, bi = False, hid per head = 5, num_heads = 2, num_fil = 7, lr = 0.01
Early stopping at best epoch:  462
best_loss = [92.47051, 111.28162, 83.67118]
time:  216.2911651134491



Pooling: 1 layers, Nonpooling: 2 layers, split by 0.5 min
rnn = LSTM, bi = False, hid per head = 5, num_heads = 2, num_fil = 3, lr = 0.0001
Early stopping at best epoch:  1543
best_loss = [143.5368, 88.26108, 83.81688]
time:  387.91629242897034



rnn = LSTM, bi = False, hid per head = 5, num_heads = 2, num_fil = 3, lr = 0.001
Early stopping at best epoch:  156
best_loss = [165.7602, 97.33017, 78.817215]
time:  177.73415184020996



rnn = LSTM, bi = False, hid per head = 5, num_heads = 2, num_fil = 3, lr = 0

Early stopping at best epoch:  2780
best_loss = [575.22076, 523.77185, 516.37976]
time:  612.6862382888794



rnn = LSTM, bi = False, hid per head = 7, num_heads = 2, num_fil = 5, lr = 0.001
Ends at final epoch 10000
Best epoch: 9971
best_loss = [427.02008, 364.38287, 348.2103]
time:  1604.5408790111542



rnn = LSTM, bi = False, hid per head = 7, num_heads = 2, num_fil = 5, lr = 0.01
Early stopping at best epoch:  4494
best_loss = [278.6233, 195.65582, 159.61871]
time:  879.162210226059



rnn = LSTM, bi = False, hid per head = 7, num_heads = 2, num_fil = 7, lr = 0.0001
Early stopping at best epoch:  2408
best_loss = [147.20624, 99.090775, 69.25763]
time:  549.3493602275848



rnn = LSTM, bi = False, hid per head = 7, num_heads = 2, num_fil = 7, lr = 0.001
Early stopping at best epoch:  464
best_loss = [132.35614, 71.99834, 91.072105]
time:  237.69197463989258



rnn = LSTM, bi = False, hid per head = 7, num_heads = 2, num_fil = 7, lr = 0.01
Early stopping at best epoch:  29
best_loss

Early stopping at best epoch:  1299
best_loss = [174.81, 109.080536, 122.84556]
time:  362.27812814712524



rnn = GRU, bi = False, hid per head = 3, num_heads = 2, num_fil = 3, lr = 0.001
Early stopping at best epoch:  890
best_loss = [101.90053, 87.39165, 92.47725]
time:  299.655455827713



rnn = GRU, bi = False, hid per head = 3, num_heads = 2, num_fil = 3, lr = 0.01
Early stopping at best epoch:  12
best_loss = [222.85687, 142.0426, 109.24713]
time:  161.13138270378113



rnn = GRU, bi = False, hid per head = 3, num_heads = 2, num_fil = 5, lr = 0.0001
Early stopping at best epoch:  1649
best_loss = [175.12111, 99.49323, 75.12736]
time:  420.4299256801605



rnn = GRU, bi = False, hid per head = 3, num_heads = 2, num_fil = 5, lr = 0.001
Early stopping at best epoch:  211
best_loss = [153.39613, 102.44388, 80.93268]
time:  196.80228686332703



rnn = GRU, bi = False, hid per head = 3, num_heads = 2, num_fil = 5, lr = 0.01
Early stopping at best epoch:  70
best_loss = [115.811295, 89

Early stopping at best epoch:  1241
best_loss = [51.870224, 100.31725, 68.25627]
time:  352.25034761428833



rnn = GRU, bi = False, hid per head = 5, num_heads = 2, num_fil = 7, lr = 0.01
Early stopping at best epoch:  58
best_loss = [140.27083, 87.99559, 61.707527]
time:  166.37404823303223



Pooling: 2 layers, Nonpooling: 2 layers, split by 0.5 min
rnn = GRU, bi = False, hid per head = 5, num_heads = 2, num_fil = 3, lr = 0.0001
Early stopping at best epoch:  452
best_loss = [239.1518, 98.82385, 120.31485]
time:  233.3724639415741



rnn = GRU, bi = False, hid per head = 5, num_heads = 2, num_fil = 3, lr = 0.001
Early stopping at best epoch:  1495
best_loss = [59.489452, 79.11825, 122.23363]
time:  401.2909667491913



rnn = GRU, bi = False, hid per head = 5, num_heads = 2, num_fil = 3, lr = 0.01
Early stopping at best epoch:  20
best_loss = [207.18927, 139.93158, 100.36935]
time:  164.05578088760376



rnn = GRU, bi = False, hid per head = 5, num_heads = 2, num_fil = 5, lr = 0.0001

Early stopping at best epoch:  300
best_loss = [121.85976, 82.63171, 93.429535]
time:  214.05271983146667



rnn = GRU, bi = False, hid per head = 7, num_heads = 2, num_fil = 5, lr = 0.01
Early stopping at best epoch:  12
best_loss = [207.4613, 125.01913, 102.46086]
time:  167.63038992881775



rnn = GRU, bi = False, hid per head = 7, num_heads = 2, num_fil = 7, lr = 0.0001
Early stopping at best epoch:  2780
best_loss = [574.6302, 523.109, 515.7208]
time:  639.1927766799927



rnn = GRU, bi = False, hid per head = 7, num_heads = 2, num_fil = 7, lr = 0.001
Ends at final epoch 10000
Best epoch: 9971
best_loss = [426.54504, 363.81967, 347.62262]
time:  1686.9951379299164



rnn = GRU, bi = False, hid per head = 7, num_heads = 2, num_fil = 7, lr = 0.01
Early stopping at best epoch:  4494
best_loss = [278.62485, 195.65411, 159.61794]
time:  930.622790813446



Pooling: 1 layers, Nonpooling: 1 layers, split by 0.5 min
rnn = RNN, bi = False, hid per head = 3, num_heads = 2, num_fil = 3, lr =

Early stopping at best epoch:  389
best_loss = [123.24864, 94.1078, 98.15368]
time:  207.3010528087616



rnn = RNN, bi = False, hid per head = 5, num_heads = 2, num_fil = 3, lr = 0.01
Early stopping at best epoch:  150
best_loss = [76.500404, 101.68584, 102.482346]
time:  171.96114945411682



rnn = RNN, bi = False, hid per head = 5, num_heads = 2, num_fil = 5, lr = 0.0001
Early stopping at best epoch:  1426
best_loss = [135.54802, 79.348755, 81.0497]
time:  363.9353437423706



rnn = RNN, bi = False, hid per head = 5, num_heads = 2, num_fil = 5, lr = 0.001
Early stopping at best epoch:  290
best_loss = [123.82084, 84.66414, 75.60043]
time:  193.1323823928833



rnn = RNN, bi = False, hid per head = 5, num_heads = 2, num_fil = 5, lr = 0.01
Early stopping at best epoch:  1131
best_loss = [108.67777, 86.608406, 89.648254]
time:  320.35527062416077



rnn = RNN, bi = False, hid per head = 5, num_heads = 2, num_fil = 7, lr = 0.0001
Early stopping at best epoch:  2780
best_loss = [574.5669

Early stopping at best epoch:  1896
best_loss = [25.929264, 81.31758, 84.46973]
time:  432.0268099308014



Pooling: 1 layers, Nonpooling: 2 layers, split by 0.5 min
rnn = RNN, bi = False, hid per head = 7, num_heads = 2, num_fil = 3, lr = 0.0001
Early stopping at best epoch:  3795
best_loss = [81.97388, 69.79177, 101.50214]
time:  718.4978122711182



rnn = RNN, bi = False, hid per head = 7, num_heads = 2, num_fil = 3, lr = 0.001
Early stopping at best epoch:  185
best_loss = [135.73048, 92.167404, 78.513824]
time:  164.17911410331726



rnn = RNN, bi = False, hid per head = 7, num_heads = 2, num_fil = 3, lr = 0.01
Early stopping at best epoch:  53
best_loss = [119.7401, 113.35747, 96.29677]
time:  146.27284741401672



rnn = RNN, bi = False, hid per head = 7, num_heads = 2, num_fil = 5, lr = 0.0001
Early stopping at best epoch:  1209
best_loss = [150.21889, 98.65039, 63.581383]
time:  309.090371131897



rnn = RNN, bi = False, hid per head = 7, num_heads = 2, num_fil = 5, lr = 0.001


In [22]:
# Hyperparameters
num_vars = train_features.size()[1]

modelname = 'RNN_TA_CA_1DCNN'
rnns = ['LSTM', 'GRU', 'RNN']
hids = [3, 5, 7]
num_time = max_time
num_cycles = 100
ep_pats = [[10000, 1000]]
lrs = [1e-4, 1e-3, 1e-2]

num_fils = [3, 5, 7]
pools = [1, 2]
npools = [1, 2]
fsize2 = 3
psize2 = 2
mids = [8, 4, 1]

nh2s = [1]

import time

for [n_ep, patience] in ep_pats:
    print(f"epoch = {n_ep}, patience = {patience}")
    trdl = globals()[f'reg_train_dataloader_{num_cycles}']
    vdl = globals()[f'reg_val_dataloader_{num_cycles}']
    tedl = globals()[f'reg_test_dataloader_{num_cycles}']
    for rnn1, hid1 in itertools.product(rnns, hids):
        bi1 = False
        
        if rnn1.startswith('Bi'):
            bi1 = True
        for pool2, npool2 in itertools.product(pools, npools):
            print(f'Pooling: {pool2} layers, Nonpooling: {npool2} layers, split by {60/max_time} min')
            for fil2, lr in itertools.product(num_fils, lrs):
                for nh2 in nh2s:
                    print(f"rnn = {rnn1}, bi = {bi1}, hid per head = {hid1}, num_heads = {nh2}, num_fil = {fil2}, lr = {lr}")
                    
                    all_hid1 = hid1 * nh2
                    
                    start = time.time()
                    
                    # Construct CNN
                    model = globals()[modelname](num_vars, num_time, num_cycles, rnn1, bi1, all_hid1, nh2, fil2, pool2, npool2, fsize2, psize2, mids).to(device)

                    # Train
                    model_train = train(model, trdl, vdl, tedl, n_ep, lr, patience, verbose = False)
                    best_model, best_loss, history, ta, ca = model_train.TrainModel()

                    print('best_loss = '+ str(best_loss))

                    history_add, state_dict_add, ta_add, ca_add = history_state_dict_add(num_cycles, modelname, n_ep, patience,
                                                                                rnn1, hid1, nh2, fil2, pool2, npool2, fsize2, psize2, lr)

                    # saving best_epoch, loss history, ta score
                    save_data(history, history_add)
                    save_data(best_model.state_dict(), state_dict_add)
                    save_data(ta, ta_add)
                    save_data(ca, ca_add)

                    print("time: ", time.time()-start)
                    print("\n\n")

epoch = 10000, patience = 1000
Pooling: 1 layers, Nonpooling: 1 layers, split by 0.5 min
rnn = LSTM, bi = False, hid per head = 3, num_heads = 1, num_fil = 3, lr = 0.0001
Early stopping at best epoch:  1615
best_loss = [153.10837, 96.9334, 93.34873]
time:  364.0830864906311



rnn = LSTM, bi = False, hid per head = 3, num_heads = 1, num_fil = 3, lr = 0.001
Early stopping at best epoch:  903
best_loss = [105.995255, 72.31671, 91.0157]
time:  265.4612100124359



rnn = LSTM, bi = False, hid per head = 3, num_heads = 1, num_fil = 3, lr = 0.01
Early stopping at best epoch:  342
best_loss = [86.67643, 92.12924, 89.27492]
time:  184.70102763175964



rnn = LSTM, bi = False, hid per head = 3, num_heads = 1, num_fil = 5, lr = 0.0001
Early stopping at best epoch:  1543
best_loss = [163.516, 94.959724, 102.54577]
time:  349.6010274887085



rnn = LSTM, bi = False, hid per head = 3, num_heads = 1, num_fil = 5, lr = 0.001
Early stopping at best epoch:  621
best_loss = [113.96022, 96.513855, 70.391

Early stopping at best epoch:  806
best_loss = [191.77065, 99.34175, 108.62135]
time:  252.43191576004028



rnn = LSTM, bi = False, hid per head = 5, num_heads = 1, num_fil = 7, lr = 0.001
Early stopping at best epoch:  191
best_loss = [148.82428, 89.328735, 95.411606]
time:  166.47958374023438



rnn = LSTM, bi = False, hid per head = 5, num_heads = 1, num_fil = 7, lr = 0.01
Early stopping at best epoch:  862
best_loss = [93.5977, 67.432655, 75.23769]
time:  260.80698680877686



Pooling: 1 layers, Nonpooling: 2 layers, split by 0.5 min
rnn = LSTM, bi = False, hid per head = 5, num_heads = 1, num_fil = 3, lr = 0.0001
Early stopping at best epoch:  2278
best_loss = [167.69261, 102.80661, 104.59839]
time:  470.1365575790405



rnn = LSTM, bi = False, hid per head = 5, num_heads = 1, num_fil = 3, lr = 0.001
Early stopping at best epoch:  371
best_loss = [141.988, 98.89814, 75.85106]
time:  196.5011022090912



rnn = LSTM, bi = False, hid per head = 5, num_heads = 1, num_fil = 3, lr = 0.

Early stopping at best epoch:  1743
best_loss = [172.37947, 107.295105, 95.31301]
time:  401.0652515888214



rnn = LSTM, bi = False, hid per head = 7, num_heads = 1, num_fil = 5, lr = 0.001
Early stopping at best epoch:  211
best_loss = [183.41919, 101.19579, 73.482025]
time:  173.92416071891785



rnn = LSTM, bi = False, hid per head = 7, num_heads = 1, num_fil = 5, lr = 0.01
Early stopping at best epoch:  2115
best_loss = [29.12132, 99.362595, 93.659805]
time:  448.1587998867035



rnn = LSTM, bi = False, hid per head = 7, num_heads = 1, num_fil = 7, lr = 0.0001
Early stopping at best epoch:  806
best_loss = [186.28061, 103.446976, 136.42538]
time:  263.4555013179779



rnn = LSTM, bi = False, hid per head = 7, num_heads = 1, num_fil = 7, lr = 0.001
Early stopping at best epoch:  453
best_loss = [105.51228, 94.559875, 99.0717]
time:  212.24955892562866



rnn = LSTM, bi = False, hid per head = 7, num_heads = 1, num_fil = 7, lr = 0.01
Early stopping at best epoch:  66
best_loss = [14

Early stopping at best epoch:  1209
best_loss = [194.58894, 125.47059, 107.42563]
time:  310.2915644645691



rnn = GRU, bi = False, hid per head = 3, num_heads = 1, num_fil = 3, lr = 0.001
Early stopping at best epoch:  185
best_loss = [169.1265, 134.12094, 93.0984]
time:  164.7492163181305



rnn = GRU, bi = False, hid per head = 3, num_heads = 1, num_fil = 3, lr = 0.01
Early stopping at best epoch:  61
best_loss = [130.73428, 115.63974, 95.28836]
time:  147.49663519859314



rnn = GRU, bi = False, hid per head = 3, num_heads = 1, num_fil = 5, lr = 0.0001
Early stopping at best epoch:  561
best_loss = [222.25555, 115.878525, 120.75035]
time:  219.2453601360321



rnn = GRU, bi = False, hid per head = 3, num_heads = 1, num_fil = 5, lr = 0.001
Early stopping at best epoch:  1359
best_loss = [37.45502, 97.99321, 93.291245]
time:  332.14610052108765



rnn = GRU, bi = False, hid per head = 3, num_heads = 1, num_fil = 5, lr = 0.01
Early stopping at best epoch:  4494
best_loss = [278.62357

Early stopping at best epoch:  301
best_loss = [96.5369, 92.99373, 99.106606]
time:  183.49658060073853



rnn = GRU, bi = False, hid per head = 5, num_heads = 1, num_fil = 7, lr = 0.01
Early stopping at best epoch:  2504
best_loss = [53.452526, 126.48035, 116.18501]
time:  493.34068036079407



Pooling: 2 layers, Nonpooling: 2 layers, split by 0.5 min
rnn = GRU, bi = False, hid per head = 5, num_heads = 1, num_fil = 3, lr = 0.0001
Early stopping at best epoch:  1769
best_loss = [177.53429, 105.66798, 84.02388]
time:  396.90847849845886



rnn = GRU, bi = False, hid per head = 5, num_heads = 1, num_fil = 3, lr = 0.001
Early stopping at best epoch:  1956
best_loss = [49.39886, 73.95324, 93.330414]
time:  423.49153542518616



rnn = GRU, bi = False, hid per head = 5, num_heads = 1, num_fil = 3, lr = 0.01
Early stopping at best epoch:  358
best_loss = [56.777344, 118.0221, 71.8793]
time:  194.8827199935913



rnn = GRU, bi = False, hid per head = 5, num_heads = 1, num_fil = 5, lr = 0.0001

Early stopping at best epoch:  1117
best_loss = [71.30524, 103.57424, 125.70482]
time:  311.8958034515381



rnn = GRU, bi = False, hid per head = 7, num_heads = 1, num_fil = 5, lr = 0.01
Early stopping at best epoch:  10
best_loss = [189.35193, 126.37448, 80.36473]
time:  149.23451018333435



rnn = GRU, bi = False, hid per head = 7, num_heads = 1, num_fil = 7, lr = 0.0001
Early stopping at best epoch:  890
best_loss = [162.19267, 97.21279, 97.84944]
time:  283.28422832489014



rnn = GRU, bi = False, hid per head = 7, num_heads = 1, num_fil = 7, lr = 0.001
Early stopping at best epoch:  211
best_loss = [162.07867, 92.65351, 74.222]
time:  181.38664484024048



rnn = GRU, bi = False, hid per head = 7, num_heads = 1, num_fil = 7, lr = 0.01
Early stopping at best epoch:  240
best_loss = [34.989864, 112.80621, 86.34647]
time:  185.89521288871765



Pooling: 1 layers, Nonpooling: 1 layers, split by 0.5 min
rnn = RNN, bi = False, hid per head = 3, num_heads = 1, num_fil = 3, lr = 0.0001
Ea

Early stopping at best epoch:  1473
best_loss = [86.17104, 90.21024, 100.62794]
time:  340.0384187698364



rnn = RNN, bi = False, hid per head = 5, num_heads = 1, num_fil = 3, lr = 0.01
Early stopping at best epoch:  928
best_loss = [129.14978, 82.49051, 99.851105]
time:  265.09637212753296



rnn = RNN, bi = False, hid per head = 5, num_heads = 1, num_fil = 5, lr = 0.0001
Early stopping at best epoch:  1144
best_loss = [211.80336, 98.28442, 86.70432]
time:  298.5766131877899



rnn = RNN, bi = False, hid per head = 5, num_heads = 1, num_fil = 5, lr = 0.001
Early stopping at best epoch:  459
best_loss = [134.71542, 130.0862, 85.326836]
time:  202.54843544960022



rnn = RNN, bi = False, hid per head = 5, num_heads = 1, num_fil = 5, lr = 0.01
Early stopping at best epoch:  20
best_loss = [195.41225, 120.749405, 76.40212]
time:  141.06029343605042



rnn = RNN, bi = False, hid per head = 5, num_heads = 1, num_fil = 7, lr = 0.0001
Early stopping at best epoch:  2780
best_loss = [575.3825

Early stopping at best epoch:  48
best_loss = [154.468, 125.44389, 81.16112]
time:  154.64192295074463



Pooling: 1 layers, Nonpooling: 2 layers, split by 0.5 min
rnn = RNN, bi = False, hid per head = 7, num_heads = 1, num_fil = 3, lr = 0.0001
Early stopping at best epoch:  2678
best_loss = [111.931076, 99.792465, 105.07762]
time:  557.2297189235687



rnn = RNN, bi = False, hid per head = 7, num_heads = 1, num_fil = 3, lr = 0.001
Early stopping at best epoch:  211
best_loss = [164.6313, 114.45157, 90.72351]
time:  183.41652655601501



rnn = RNN, bi = False, hid per head = 7, num_heads = 1, num_fil = 3, lr = 0.01
Early stopping at best epoch:  548
best_loss = [38.107864, 93.43776, 97.398796]
time:  226.00990796089172



rnn = RNN, bi = False, hid per head = 7, num_heads = 1, num_fil = 5, lr = 0.0001
Early stopping at best epoch:  806
best_loss = [172.22675, 106.833954, 96.24993]
time:  261.791437625885



rnn = RNN, bi = False, hid per head = 7, num_heads = 1, num_fil = 5, lr = 0.001

In [28]:
modelname = 'RNN_TA_CA_1DCNN'
rnns = ['LSTM', 'GRU', 'RNN']
hids = [3, 5, 7]
num_time = max_time
num_cycles = 100
ep_pats = [[10000, 1000]]
lrs = [1e-4, 1e-3, 1e-2]

num_fils = [3, 5, 7]
pools = [1, 2]
npools = [1, 2]
fsize2 = 3
psize2 = 2
mids = [8, 4, 1]

nh2s = [1, 2, 3, 4, 5]

for [n_ep, patience] in ep_pats:
    for nh2 in nh2s:
        for rnn1 in rnns:
            df = pd.DataFrame(columns = ['train rmse', 'val rmse', 'test rmse'])
            bi1 = False
            if rnn1.startswith('Bi'):
                bi1 = True
            for hid1 in hids:
                for pool2, npool2 in itertools.product(pools, npools):
                    for fil2, lr in itertools.product(num_fils, lrs):
                        history_add, state_dict_add, ta_add, ca_add = history_state_dict_add(num_cycles, modelname, n_ep, patience,
                                                                                    rnn1, hid1, nh2, fil2, pool2, npool2, fsize2, psize2, lr)

                        history = load_data(history_add)

                        df.loc[f'{nh2}_{rnn1}_{hid1}_{fil2}_{pool2}_{npool2}_{fsize2}_{lr}'] = [history.iloc[-1, 0], history.iloc[-1, 1], history.iloc[-1, 2]]

            print(n_ep, patience, nh2, 'heads', rnn1)
            df
            df_add = find_directory(back_num = 0, foldername = [f'{num_cycles} cycles', f'Depth Test_col_{n_ep}_{patience}', modelname], 
                            filename = f'{nh2} heads_{modelname}_{rnn1}.csv')
            df.to_csv(df_add)

10000 1000 1 heads LSTM


,train rmse,val rmse,test rmse
1_LSTM_3_3_1_1_3_0.0001,153.108368,96.933403,93.348732
1_LSTM_3_3_1_1_3_0.001,105.995255,72.316711,91.015701
1_LSTM_3_3_1_1_3_0.01,86.676430,92.129242,89.274918
1_LSTM_3_5_1_1_3_0.0001,163.516006,94.959724,102.545769
1_LSTM_3_5_1_1_3_0.001,113.960220,96.513855,70.391159
...,...,...,...
1_LSTM_7_5_2_2_3_0.001,130.206360,96.482635,103.401283
1_LSTM_7_5_2_2_3_0.01,35.773277,78.565865,85.029610
1_LSTM_7_7_2_2_3_0.0001,221.086319,112.818222,117.906982
1_LSTM_7_7_2_2_3_0.001,149.128036,96.364151,73.352280


10000 1000 1 heads GRU


,train rmse,val rmse,test rmse
1_GRU_3_3_1_1_3_0.0001,200.632629,110.190025,91.808601
1_GRU_3_3_1_1_3_0.001,111.884705,90.715393,90.421829
1_GRU_3_3_1_1_3_0.01,32.357758,89.387375,71.651489
1_GRU_3_5_1_1_3_0.0001,202.248901,100.147263,83.360168
1_GRU_3_5_1_1_3_0.001,50.381138,99.086891,93.005981
...,...,...,...
1_GRU_7_5_2_2_3_0.001,71.305237,103.574242,125.704819
1_GRU_7_5_2_2_3_0.01,189.351929,126.374481,80.364731
1_GRU_7_7_2_2_3_0.0001,162.192673,97.212791,97.849442
1_GRU_7_7_2_2_3_0.001,162.078674,92.653511,74.222000


10000 1000 1 heads RNN


...

,train rmse,val rmse,test rmse
5_LSTM_3_3_1_1_3_0.0001,131.118042,77.675812,80.681290
5_LSTM_3_3_1_1_3_0.001,112.727837,88.196838,104.277519
5_LSTM_3_3_1_1_3_0.01,21.290863,92.756577,93.062073
5_LSTM_3_5_1_1_3_0.0001,154.887787,78.079140,84.113457
5_LSTM_3_5_1_1_3_0.001,123.108017,71.485207,77.239418
...,...,...,...
5_LSTM_7_5_2_2_3_0.001,426.731628,364.040894,347.853516
5_LSTM_7_5_2_2_3_0.01,29.008072,136.190964,76.596924
5_LSTM_7_7_2_2_3_0.0001,172.491806,97.632263,111.396545
5_LSTM_7_7_2_2_3_0.001,76.484604,91.792198,108.165672


10000 1000 5 heads GRU


,train rmse,val rmse,test rmse
5_GRU_3_3_1_1_3_0.0001,178.921188,103.948364,114.174438
5_GRU_3_3_1_1_3_0.001,176.046234,120.874268,94.706612
5_GRU_3_3_1_1_3_0.01,278.623291,195.655884,159.618744
5_GRU_3_5_1_1_3_0.0001,211.547821,98.002396,113.153641
5_GRU_3_5_1_1_3_0.001,134.697067,116.018814,55.527588
...,...,...,...
5_GRU_7_5_2_2_3_0.001,23.952662,111.992477,88.614166
5_GRU_7_5_2_2_3_0.01,51.777863,100.836014,134.303116
5_GRU_7_7_2_2_3_0.0001,213.204926,111.328476,92.953720
5_GRU_7_7_2_2_3_0.001,133.293945,117.970078,90.460312


10000 1000 5 heads RNN


,train rmse,val rmse,test rmse
5_RNN_3_3_1_1_3_0.0001,157.061310,82.910355,90.211739
5_RNN_3_3_1_1_3_0.001,118.159119,94.778137,94.979507
5_RNN_3_3_1_1_3_0.01,136.093216,90.918503,86.465523
5_RNN_3_5_1_1_3_0.0001,187.121048,86.630699,92.651566
5_RNN_3_5_1_1_3_0.001,114.570831,86.387810,75.818939
...,...,...,...
5_RNN_7_5_2_2_3_0.001,86.625008,94.992584,84.213760
5_RNN_7_5_2_2_3_0.01,112.336884,122.586365,87.132660
5_RNN_7_7_2_2_3_0.0001,84.125168,91.893898,108.346771
5_RNN_7_7_2_2_3_0.001,23.905115,115.555061,85.146347
